In [1]:
import os
import re
import sys
from openai import AzureOpenAI
import pandas as pd
import numpy as np
# import pymssql
from dotenv import load_dotenv, find_dotenv
import warnings 
import pyodbc
import mlflow
warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

mlflow.openai.autolog()

In [2]:
os.environ.get("AZURE_OPENAI_ENDPOINT")

'https://20892-m7iyjxep-eastus2.cognitiveservices.azure.com/'

In [3]:
os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

'gpt-4.1'

In [4]:
os.environ.get("AZURE_OPENAI_VERSION")

'2025-01-01-preview'

In [5]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

# Chat Completions : Older models

In [6]:

# # Create client
# client = AzureOpenAI(
#   default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
#   api_key=API_KEY,
#   azure_endpoint=API_ENDPOINT,
#   azure_deployment= AZURE_DEPLOYMENT,
#   api_version=API_VERSION, 
# )

# # Create completions
# completion = client.chat.completions.create(
#   model=MODEL, 
#   stream=True,
#   messages=[
#     {
#       "role": "user",
#       "content": "Tell me a long joke",
#     },
#   ],
# )

# for chunk in completion:
#   try:
#     print(chunk.choices[0].delta.content or "", end="")
#   except Exception:
#     pass


In [7]:

# Create client
client = AzureOpenAI(
  default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
  api_key=API_KEY,
  azure_endpoint=API_ENDPOINT,
  azure_deployment= AZURE_DEPLOYMENT,
  api_version=API_VERSION, 
)


In [8]:

# Create completions
completion = client.chat.completions.create(
  model=MODEL, 
  stream=True,
  messages=[
    {
      "role": "user",
      "content": "Tell me a long joke",
    },
  ],
)

for chunk in completion:
  try:
    print(chunk.choices[0].delta.content or "", end="")
  except Exception:
    pass


2025/06/16 09:33:42 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Absolutely! Here’s a long joke for you. Sit back and enjoy:

---

A man is driving down a country road when he sees a sign:  
TALKING DOG FOR SALE – INQUIRE WITHIN

Curious, he pulls in, walks up to the farmhouse, and knocks on the door. The owner greets him, and the man asks, “I saw your sign about a talking dog; is that for real?”

The farmer nods and points to the backyard. The man steps through the gate, and sure enough, there’s an old dog sitting calmly on the porch. He hesitates, then says, “Excuse me… are you the talking dog?”

The dog looks up and says, “Yep, that’s me.”

The man’s jaw drops. “What? You can really talk?!”

The dog nods. “Absolutely. I discovered it when I was young. It got me recruited by the CIA. I traveled the world, sat in on high-level meetings—no one ever suspected a dog was fluent in multiple languages. After that, I settled down here to relax. Now I just guard the porch, take naps, and give belly rubs to myself.”

The man, stunned, runs back to the farme

In [ ]:
def generate_response(prompt):
    completions = client.chat.completions.create(
        model=MODEL,
        stream=True,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    for chunk in completions:
        try:
            # print(chunk.choices[0].delta.content or "", end="")
            print(chunk.choices[0].delta.content or "", end="")
        except Exception:
            pass

In [ ]:
print(generate_response("What makes AI agents AI agents?"))

Great question! To answer **what makes AI agents 'AI agents'**, let's break it down:

### 1. **What is an Agent?**
In artificial intelligence (AI), **an agent is anything that perceives its environment and acts upon it**. It can be as simple as a thermostat (perceives temperature, turns heater on/off) or as complex as an autonomous car.

### 2. **What makes an Agent an AI Agent?**
An **AI agent** is specifically a *computational* agent that uses **artificial intelligence techniques** to operate. This means:

- **Perceives**: It gets input from the environment (via sensors or data).
- **Thinks/Reasons**: It processes that input intelligently (using logic, learning, planning, etc).
- **Acts**: It acts back on the environment (via effectors or commands).
- **Goal-Oriented**: It typically works towards a goal, optimizing its actions to succeed.

### 3. **Key Features That Make an Agent an AI Agent**
- **Autonomy**: Makes at least some decisions on its own, with limited human intervention.


# SQL Queries

In [ ]:
# to the RSVR
server_reservior = os.environ.get("rsvr")
rsvr_password = os.environ.get("rsvr_password")
rsvr_user =  os.environ.get("rsvr_user") #"SBICZA01\A238737"

# Connecting to the Resevoir
conn_reservior = pymssql.connect(server = server_reservior,
                                user = rsvr_user, 
                                password = rsvr_password)
cursor_rsvr = conn_reservior.cursor()

In [ ]:
# Query to get the data from the Reservoir
# df_data = pd.read_sql_query("""
#                             SELECT top 1000 * 
#                             FROM [RSVRRealtimeFinacleLS].[PLSFIN].[vwCuratedLSOFinacleTransactionsHistoryImpl] WITH (NOLOCK)
#                             """, conn_reservior)
# cursor_rsvr.close()
# conn_reservior.close()

# df_data.head()

In [ ]:
import re
import pandas as pd
import pymssql
import os
from tqdm import tqdm

def validate_sql_query(query):
    """
    Validates SQL query to ensure it's safe and only contains SELECT statements.
    Returns (is_valid, error_message).
    """
    import re
    
    # Normalize query - remove comments and extra whitespace
    normalized_query = re.sub(r'--.*', '', query)  # Remove line comments
    normalized_query = re.sub(r'/\*.*?\*/', '', normalized_query, flags=re.DOTALL)  # Remove block comments
    normalized_query = ' '.join(normalized_query.split()).upper()  # Normalize whitespace and convert to uppercase
    
    # List of dangerous SQL keywords that should be blocked
    dangerous_keywords = [
        'INSERT', 'UPDATE', 'DELETE', 'DROP', 'CREATE', 'ALTER', 'TRUNCATE',
        'MERGE', 'REPLACE', 'EXEC', 'EXECUTE', 'CALL', 'DECLARE', 'SET',
        'GRANT', 'REVOKE', 'COMMIT', 'ROLLBACK', 'SAVEPOINT', 'BEGIN',
        'TRANSACTION', 'LOCK', 'UNLOCK', 'BACKUP', 'RESTORE', 'SHUTDOWN',
        'USE', 'SHOW', 'DESCRIBE', 'EXPLAIN', 'LOAD', 'OUTFILE', 'INFILE',
        'IMPORT', 'EXPORT', 'BULK', 'OPENROWSET', 'OPENDATASOURCE'
    ]
    
    # Check for dangerous keywords
    for keyword in dangerous_keywords:
        if re.search(r'\b' + keyword + r'\b', normalized_query):
            return False, f"Dangerous SQL keyword detected: {keyword}"
    
    # Ensure query starts with SELECT
    if not normalized_query.strip().startswith('SELECT'):
        return False, "Only SELECT statements are allowed"
    
    # Check for suspicious patterns
    suspicious_patterns = [
        r';\s*(INSERT|UPDATE|DELETE|DROP|CREATE|ALTER)',  # Multiple statements
        r'UNION.*?(INSERT|UPDATE|DELETE|DROP|CREATE|ALTER)',  # Union injection
        r'OR\s+1\s*=\s*1',  # SQL injection pattern
        r'AND\s+1\s*=\s*1',  # SQL injection pattern
        r"'.*?'.*?;",  # Potential injection with semicolon
    ]
    
    for pattern in suspicious_patterns:
        if re.search(pattern, normalized_query, re.IGNORECASE):
            return False, f"Suspicious SQL pattern detected: {pattern}"
    
    return True, "Query is valid"

def execute_multiple_sql_code(sql_code, connection=None, connection_type="mssql"):
    """
    Accepts a string containing one or more SQL queries, each enclosed in triple backticks.
    Executes each query sequentially and stores the results in a list.
    
    Parameters:
    - sql_code: String containing SQL queries in ```sql code blocks
    - connection: Database connection object (pymssql, sqlite3, etc.)
    - connection_type: Type of database connection ("mssql", "sqlite", "mysql", "postgresql")
    
    Returns a list of results (SELECT returns DataFrame as JSON, validation errors return error messages).
    """
    import pandas as pd
    import json
    import re
    
    if connection is None:
        return [{"query": "", "result": "Error: No database connection provided", "status": "connection_error"}]
    
    # Improved regex to handle various backtick formats
    # This covers both ```sql and ``` sql (with spaces) and handles multiline blocks better
    queries = re.findall(r'```\s*sql\s*(.*?)```', sql_code, re.DOTALL)
    results = []
    
    if not queries:
        # Try more lenient pattern (without sql keyword)
        queries = re.findall(r'```(.*?)```', sql_code, re.DOTALL)
        
        if not queries:
            return [{"query": sql_code, 
                    "result": "No SQL queries found in the provided code. Please format queries in ```sql code blocks.",
                    "status": "format_error"}]
    
    for query_idx, query in enumerate(queries):
        query = query.strip()
        if not query:
            results.append({
                "query": "", 
                "result": f"Empty query found in code block #{query_idx+1}",
                "status": "validation_error"
            })
            continue
            
        # Validate the query first
        is_valid, validation_message = validate_sql_query(query)
        
        if not is_valid:
            results.append({
                "query": query, 
                "result": f"Query validation failed: {validation_message}",
                "status": "validation_error"
            })
            continue
        
        try:
            # Execute only SELECT statements (already validated)
            print(f"Running Query #{query_idx+1} ______________________")
            print(f"Executing query: {query[:100]}{'...' if len(query) > 100 else ''}")
            
            # Execute query
            df = pd.read_sql_query(query, connection)
            
            print(f"Query executed successfully - returned {len(df)} rows with {len(df.columns)} columns")
            
            # Handle empty result sets gracefully
            if len(df) == 0:
                results.append({
                    "query": query,
                    "result": "[]",  # Empty JSON array
                    "status": "success",
                    "row_count": 0,
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
                continue
            
            # Convert to JSON with error handling
            try:
                json_result = df.to_json(orient='records', date_format='iso')
                results.append({
                    "query": query, 
                    "result": json_result,
                    "status": "success",
                    "row_count": len(df),
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
            except Exception as json_err:
                # Fallback for JSON conversion errors
                results.append({
                    "query": query, 
                    "result": f"Error converting results to JSON: {str(json_err)}",
                    "status": "json_error",
                    "row_count": len(df),
                    "column_info": {col: str(dtype) for col, dtype in df.dtypes.items()}
                })
                
        except Exception as e:
            results.append({
                "query": query, 
                "result": f"Error executing SQL: {str(e)}",
                "status": "execution_error"
            })
    
    return results

def execute_sql_with_mssql_connection(sql_code):
    """
    Execute SQL queries using the existing MSSQL connection setup.
    """
    try:
        # Use your existing connection setup
        server_reservior = os.environ.get("rsvr")
        rsvr_password = os.environ.get("rsvr_password")
        rsvr_user = os.environ.get("rsvr_user")
        
        if not all([server_reservior, rsvr_password, rsvr_user]):
            return [{"query": "", 
                    "result": "Missing environment variables for database connection. Check rsvr, rsvr_password, rsvr_user", 
                    "status": "configuration_error"}]
        
        conn_reservior = pymssql.connect(
            server=server_reservior,
            user=rsvr_user, 
            password=rsvr_password
        )
        
        # Execute the queries
        results = execute_multiple_sql_code(sql_code, conn_reservior, "mssql")
        
        # Close connection
        conn_reservior.close()
        
        return results
        
    except Exception as e:
        return [{"query": "", "result": f"Connection error: {str(e)}", "status": "connection_error"}]

In [ ]:
def execute_multiple_sql_code(sql_code, connection=None, connection_type="mssql", use_raw_cursor=False):
    import pandas as pd
    import json
    import re

    if connection is None:
        return [{"query": "", "result": "Error: No database connection provided", "status": "connection_error"}]

    queries = re.findall(r'```\s*sql\s*(.*?)```', sql_code, re.DOTALL)
    results = []

    if not queries:
        queries = re.findall(r'```(.*?)```', sql_code, re.DOTALL)
        if not queries:
            return [{"query": sql_code, 
                     "result": "No SQL queries found in the provided code. Please format queries in ```sql code blocks.",
                     "status": "format_error"}]

    for query_idx, query in enumerate(queries):
        query = query.strip()
        if not query:
            results.append({
                "query": "", 
                "result": f"Empty query found in code block #{query_idx+1}",
                "status": "validation_error"
            })
            continue

        is_valid, validation_message = validate_sql_query(query)
        if not is_valid:
            results.append({
                "query": query, 
                "result": f"Query validation failed: {validation_message}",
                "status": "validation_error"
            })
            continue

        try:
            print(f"Running Query #{query_idx+1} ______________________")
            print(f"Executing query: {query[:100]}{'...' if len(query) > 100 else ''}")

            if use_raw_cursor:
                cursor = connection.cursor()
                cursor.execute(query)
                columns = [desc[0] for desc in cursor.description]
                rows = cursor.fetchall()
                df = pd.DataFrame(rows, columns=columns)
            else:
                df = pd.read_sql_query(query, connection)

            if len(df) == 0:
                results.append({
                    "query": query,
                    "result": "[]",
                    "status": "success",
                    "row_count": 0,
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
                continue

            try:
                json_result = df.to_json(orient='records', date_format='iso')
                results.append({
                    "query": query, 
                    "result": json_result,
                    "status": "success",
                    "row_count": len(df),
                    "column_count": len(df.columns),
                    "columns": df.columns.tolist()
                })
            except Exception as json_err:
                results.append({
                    "query": query, 
                    "result": f"Error converting results to JSON: {str(json_err)}",
                    "status": "json_error",
                    "row_count": len(df),
                    "column_info": {col: str(dtype) for col, dtype in df.dtypes.items()}
                })

        except Exception as e:
            results.append({
                "query": query, 
                "result": f"Error executing SQL: {str(e)}",
                "status": "execution_error"
            })

    return results


In [ ]:
# Example usage:
sql_code_example = """
```sql
SELECT top 100 * 
FROM [RSVRRealtimeFinacleLS].[PLSFIN].[vwCuratedLSOFinacleTransactionsHistoryImpl] WITH (NOLOCK);
```
"""


In [ ]:

# Execute the example (uncomment to test)
results = execute_sql_with_mssql_connection(sql_code_example)
for res in results:
    print(f"Status: {res['status']}")
    print(f"Query: {res['query'][:50]}...")
    if res['status'] == 'success':
        print(f"Rows returned: {res['row_count']}")
    print(f"Result: {res['result'][:200]}...")
    print("-" * 50)

Status: connection_error
Query: ...
Result: Connection error: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (DSSQLRESAG04.za.sbicdirectory.com)\n')...
--------------------------------------------------


In [ ]:
results

[{'query': '',
  'result': "Connection error: (20009, b'DB-Lib error message 20009, severity 9:\\nUnable to connect: Adaptive Server is unavailable or does not exist (DSSQLRESAG04.za.sbicdirectory.com)\\n')",
  'status': 'connection_error'}]

# LocalDB: pyodbc

In [4]:
server = os.environ.get("server_data_studio") 
database = os.environ.get("database")
password = os.environ.get("password")
username =  os.environ.get("username") #"SBICZA01\A238737"


In [5]:
server, database, username

('localhost,1433', 'master', 'SA')

In [4]:
import pyodbc

# List available drivers
print("Available drivers:")
for driver in pyodbc.drivers():
    print(f"  - {driver}")

Available drivers:
  - ODBC Driver 17 for SQL Server
  - ODBC Driver 18 for SQL Server


### Connect to satabase

In [2]:
# import connect_to_sql_server module
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

try:
    from db.sql_connector import connect_to_sql_server
    from db.sql_query_executor import (validate_sql_query,
                                       execute_multiple_sql_code,
                                       execute_sql_with_pyodbc)
    print("Successfully imported")
except ImportError as e:
    print(f"Error importing: {e}")
    print("Ensure the file/s is in the parent directory of your current script.")
    sys.exit(1)


Successfully imported


In [6]:
# Example usage:
# Windows authentication
# conn, cursor = connect_to_sql_server(server, database, auth_type='windows')

# SQL Server authentication
conn, cursor = connect_to_sql_server(server, database, auth_type='sql', username= username, password=password)

# Don't forget to close the connection when done
# if conn:
#     conn.close()

Available SQL Server drivers: ODBC Driver 18 for SQL Server, ODBC Driver 17 for SQL Server
Attempting connection: sql auth using ODBC Driver 18 for SQL Server
Server: localhost,1433, Database: master
✓ Successfully connected to SQL Server
  Server: localhost,1433
  Database: master
  Authentication: sql
  Driver: ODBC Driver 18 for SQL Server


In [7]:
try:
    query = """
    SELECT TOP 100 * 
    FROM [master].[dbo].[customer_information] WITH (NOLOCK);
    """
    df_pandas = pd.read_sql_query(query, conn)
    display(df_pandas.head())
except Exception as e:
    print(f"Error executing query: {e}")

,id,full_name,email,phone_number,address,account_number,account_type,balance,gender,age,...,loan_amount_applied_for,loan_status,loan_tenure,loan_interest_rate,loan_purpose,loan_application_date,last_login,product_holding,no_product_holding,entrenchment_score
0,10275214,Chan Yu Smith,Chan.Yu@chocho.com,+24-222-444-6662,"43 Dobstar, Kyalami, JHB, 7777",9633303278,Cheque,28914.04,Male,70,...,0.00,None,NaN,NaN,None,NaT,2024-07-09 14:42:54,"[""Agricultural Loans"", ""Business Credit Card"",...",11,47.83
1,11584657,Jimmy Jimmy,Jimmy@spades.co.za,919.775.2040x0560,"876 Spades, JHB 2345",5760482938,Savings,24827.17,Male,26,...,1000.00,Pending,32.0,10.75,Education,2025-05-31 08:50:15.317,2024-06-18 04:27:58,"[""Platinum Credit Card"", ""Checking Account"", ""...",4,17.39
2,11965767,Kelly Moreno,bdonaldson@example.org,(653)950-6390x36269,"9636 White Mall, Lake Laura, ND 12630",9093175900,Business,27744.50,Female,18,...,21231.81,Approved,33.0,12.49,Personal,2024-11-19 10:30:27.000,2025-04-01 23:50:51,"[""Classic Credit Card"", ""Savings Account"", ""Bu...",4,17.39
3,13132302,Gina Ho,uhoffman@example.org,+1-741-525-3160x47841,"144 Wallace Tunnel Apt. 491, Nicholastown, GU ...",2939948636,Savings,16704.52,Female,71,...,0.00,None,NaN,NaN,None,NaT,2024-11-25 15:42:29,"[""Commercial Lines of Credit"", ""Gold Credit Ca...",5,21.74
4,14613766,Stephanie White,davidtodd@example.com,375-238-5391x05169,"232 Stephanie Landing Apt. 387, Murphytown, CT...",7042128924,Business,23568.48,Male,20,...,10000.00,Pending,32.0,10.17,Education,NaT,2024-06-18 02:37:29,"[""Home Mortgages"", ""Commercial Lines of Credit...",3,13.04


In [8]:
try:
    query = """
    SELECT TOP 100 * 
    FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
    """
    df_pandas = pd.read_sql_query(query, conn)
    display(df_pandas.head())
except Exception as e:
    print(f"Error executing query: {e}")

,transaction_id,customer_id,transaction_date,transaction_type,amount,currency,description,category,channel,status,reference_number,merchant_name,merchant_category,account_from,account_to,balance_after,location,created_at,updated_at
0,12393752,28918917,2023-10-01 09:48:10,Payment,-2744.16,ZAR,Payment - Shopping,Shopping,ACH,Completed,REF2626OK8897,Gutierrez-Jones,Shopping,6933192193,None,10199.27,"Williamside, Dominica",2025-06-16 09:59:24,2025-06-16 09:59:24
1,63798465,51829432,2023-10-06 10:23:20,Withdrawal,-3316.33,ZAR,Cash withdrawal - Direct Deposit,Travel,Direct Deposit,Pending,REF1171On3662,None,None,5038143544,None,25425.87,"East Stacytown, Grenada",2025-06-16 09:59:24,2025-06-16 09:59:24
2,166643323,17803343,2023-11-26 09:29:37,Deposit,22626.52,ZAR,Deposit - POS Terminal,Refund,POS Terminal,Failed,REF2863zu6711,None,None,8318791786,None,43070.97,"West Chelseamouth, Falkland Islands (Malvinas)",2025-06-16 09:59:23,2025-06-16 09:59:23
3,291217550,26194287,2025-01-29 04:51:14,Deposit,8797.42,ZAR,Deposit - Wire Transfer,Banking,Wire Transfer,Failed,REF1328yU7424,None,None,9624591121,None,1949.03,"New Amber, Pakistan",2025-06-16 09:59:23,2025-06-16 09:59:23
4,379218827,43230613,2024-05-20 19:24:41,Deposit,17794.41,USD,Deposit - Wire Transfer,Education,Wire Transfer,Completed,REF9787XX3528,None,None,3351627289,None,6204.75,"Kimberlyberg, Cameroon",2025-06-16 09:59:23,2025-06-16 09:59:23


In [9]:
try:# Option 2: Using cursor directly
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    # Create DataFrame from cursor results
    df_cursor = pd.DataFrame.from_records(rows, columns=columns)
    # Display the DataFrame
    display(df_cursor.head())
except Exception as e:
    print(f"Error executing query with cursor: {e}")

,transaction_id,customer_id,transaction_date,transaction_type,amount,currency,description,category,channel,status,reference_number,merchant_name,merchant_category,account_from,account_to,balance_after,location,created_at,updated_at
0,12393752,28918917,2023-10-01 09:48:10,Payment,-2744.16,ZAR,Payment - Shopping,Shopping,ACH,Completed,REF2626OK8897,Gutierrez-Jones,Shopping,6933192193,None,10199.27,"Williamside, Dominica",2025-06-16 09:59:24,2025-06-16 09:59:24
1,63798465,51829432,2023-10-06 10:23:20,Withdrawal,-3316.33,ZAR,Cash withdrawal - Direct Deposit,Travel,Direct Deposit,Pending,REF1171On3662,None,None,5038143544,None,25425.87,"East Stacytown, Grenada",2025-06-16 09:59:24,2025-06-16 09:59:24
2,166643323,17803343,2023-11-26 09:29:37,Deposit,22626.52,ZAR,Deposit - POS Terminal,Refund,POS Terminal,Failed,REF2863zu6711,None,None,8318791786,None,43070.97,"West Chelseamouth, Falkland Islands (Malvinas)",2025-06-16 09:59:23,2025-06-16 09:59:23
3,291217550,26194287,2025-01-29 04:51:14,Deposit,8797.42,ZAR,Deposit - Wire Transfer,Banking,Wire Transfer,Failed,REF1328yU7424,None,None,9624591121,None,1949.03,"New Amber, Pakistan",2025-06-16 09:59:23,2025-06-16 09:59:23
4,379218827,43230613,2024-05-20 19:24:41,Deposit,17794.41,USD,Deposit - Wire Transfer,Education,Wire Transfer,Completed,REF9787XX3528,None,None,3351627289,None,6204.75,"Kimberlyberg, Cameroon",2025-06-16 09:59:23,2025-06-16 09:59:23


## Multiple SQL Queries Execution

In [8]:
# Example usage:
sql_code_example = """
```sql
SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
```
```sql
SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
```

"""

# Execute the example (uncomment to test)
results = execute_sql_with_pyodbc(sql_code_example)
for res in results:
    print(f"Status: {res['status']}")
    print(f"Query: {res['query'][:100]}...")
    if res['status'] == 'success':
        print(f"Rows returned: {res['row_count']}")
    print(f"Result: {res['result'][:200]}...")
    print("-" * 50)

Loaded environment variables from /Users/mzwandilemhlongo/Desktop/Data Science/PersonalProjects/rag-system/rag-text2sql/.env
Available SQL Server drivers: ODBC Driver 18 for SQL Server, ODBC Driver 17 for SQL Server
Attempting connection: sql auth using ODBC Driver 18 for SQL Server
Server: localhost,1433, Database: master
✓ Successfully connected to SQL Server
  Server: localhost,1433
  Database: master
  Authentication: sql
  Driver: ODBC Driver 18 for SQL Server
Running Query #1 ______________________
Executing query: SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);


Query executed successfully - returned 1000 rows with 19 columns
Running Query #2 ______________________
Executing query: SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);


Query executed successfully - returned 5001 rows with 1 columns
Status: success
Query: SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);...
Rows returned: 1

In [9]:
results

[{'query': 'SELECT top 1000 * \nFROM [master].[dbo].[transaction_history] WITH (NOLOCK);',
  'status': 'success',
  'row_count': 1000,
  'column_count': 19,
  'columns': ['transaction_id',
   'customer_id',
   'transaction_date',
   'transaction_type',
   'amount',
   'currency',
   'description',
   'category',
   'channel',
   'status',
   'reference_number',
   'merchant_name',
   'merchant_category',
   'account_from',
   'account_to',
   'balance_after',
   'location',
   'created_at',
   'updated_at'],
  'result': '[{"transaction_id":12393752,"customer_id":28918917,"transaction_date":"2023-10-01T09:48:10.000","transaction_type":"Payment","amount":-2744.16,"currency":"ZAR","description":"Payment - Shopping","category":"Shopping","channel":"ACH","status":"Completed","reference_number":"REF2626OK8897","merchant_name":"Gutierrez-Jones","merchant_category":"Shopping","account_from":"6933192193","account_to":null,"balance_after":10199.27,"location":"Williamside, Dominica","created_at":

In [10]:
def display_sql_results(results, max_query_preview=200, max_result_preview=100):
    """
    Display SQL query results in a formatted way.
    
    Parameters:
    - results: List of result dictionaries from execute_sql_with_pyodbc
    - max_query_preview: Maximum number of characters to show in query preview
    - max_result_preview: Maximum number of characters to show in result preview
    
    Returns:
    - None (prints results to console)
    """
    if not results:
        print("No results to display.")
        return
        
    for idx, res in enumerate(results, 1):
        # Create a separator with query number
        print(f"\n{'=' * 20} QUERY #{idx} {'=' * 20}")
        
        # Status with color indicators (using ANSI escape codes)
        status = res['status']
        status_color = {
            'success': '\033[92m',      # Green
            'validation_error': '\033[93m',  # Yellow
            'execution_error': '\033[91m',   # Red
            'connection_error': '\033[91m',  # Red
            'json_error': '\033[93m',        # Yellow
            'format_error': '\033[93m',      # Yellow
            'configuration_error': '\033[91m' # Red
        }.get(status, '')
        end_color = '\033[0m'  # Reset color
        
        print(f"Status: {status_color}{status}{end_color}")
        
        # Query preview with ellipsis if truncated
        query = res.get('query', '')
        query_preview = query[:max_query_preview]
        if len(query) > max_query_preview:
            query_preview += "..."
        print(f"Query: {query_preview}")
        
        # For successful queries, show row and column info
        if status == 'success':
            print(f"Rows: {res.get('row_count', 'N/A')} | Columns: {res.get('column_count', 'N/A')}")
            if 'columns' in res:
                print(f"Column names: {', '.join(res['columns'])}")
        
        # Result preview with ellipsis if truncated
        result = res.get('result', '')
        result_preview = result[:max_result_preview]
        if len(result) > max_result_preview:
            result_preview += "..."
        print(f"Result: {result_preview}")
        
        # End of result separator
        print("-" * 50)

In [11]:
display_sql_results(results)


==================== QUERY #1 ====================
Status: success
Query: SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
Rows: 1000 | Columns: 19
Column names: transaction_id, customer_id, transaction_date, transaction_type, amount, currency, description, category, channel, status, reference_number, merchant_name, merchant_category, account_from, account_to, balance_after, location, created_at, updated_at
Result: [{"transaction_id":12393752,"customer_id":28918917,"transaction_date":"2023-10-01T09:48:10.000","tra...
--------------------------------------------------

==================== QUERY #2 ====================
Status: success
Query: SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);
Rows: 5001 | Columns: 1
Column names: transaction_id
Result: [{"transaction_id":12393752},{"transaction_id":63798465},{"transaction_id":166643323},{"transaction_...
--------------------------------------------------


In [12]:
sql_results = execute_sql_with_pyodbc(sql_code_example)
display_sql_results(sql_results, max_query_preview=200, max_result_preview=100)

Loaded environment variables from /Users/mzwandilemhlongo/Desktop/Data Science/PersonalProjects/rag-system/rag-text2sql/.env
Available SQL Server drivers: ODBC Driver 18 for SQL Server, ODBC Driver 17 for SQL Server
Attempting connection: sql auth using ODBC Driver 18 for SQL Server
Server: localhost,1433, Database: master
✓ Successfully connected to SQL Server
  Server: localhost,1433
  Database: master
  Authentication: sql
  Driver: ODBC Driver 18 for SQL Server
Running Query #1 ______________________
Executing query: SELECT top 1000 * 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);


Query executed successfully - returned 1000 rows with 19 columns
Running Query #2 ______________________
Executing query: SELECT distinct transaction_id 
FROM [master].[dbo].[transaction_history] WITH (NOLOCK);


Query executed successfully - returned 5001 rows with 1 columns

==================== QUERY #1 ====================
Status: success
Query: SELECT top 1000 * 
FROM [master].[dbo].[tra

# Agents

In [13]:
from pathlib import Path

import os
import re
import sys
from openai import AzureOpenAI
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv, find_dotenv
import warnings 
warnings.filterwarnings("ignore")
load_dotenv(find_dotenv())

# Add the src directory to the Python path
# src_dir = str(Path(__file__).parent.parent)
# sys.path.append(src_dir)
# Add the parent directory to sys.path to allow imports from rag-compliance

True

In [14]:

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Now import the modules from rag-compliance
try:
    # Import prompts
    from prompts.prompt_agent_router import prompt_agent_router
    from prompts.prompt_agent_sql_analysis import prompt_agent_sql_analysis
    from agents.agents import routing_agent, agent_sql_analysis

    from tools.tools import tools_definitions
    
    # Import agents (if any)
    # from rag_compliance.agents.your_agent_module import your_agent_function
    
    print("Successfully imported modules from rag-compliance directory")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Make sure the directory structure is correct and modules exist")
    
    # Debugging information
    print("\nDebugging information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Parent directory added to path: {parent_dir}")
    print(f"Full sys.path: {sys.path}")
    
    # List files in the expected directories to help with debugging
    rag_compliance_dir = os.path.join(parent_dir, 'rag_compliance')
    if os.path.exists(rag_compliance_dir):
        print(f"\nContents of {rag_compliance_dir}:")
        print(os.listdir(rag_compliance_dir))
        
        prompts_dir = os.path.join(rag_compliance_dir, 'prompts')
        if os.path.exists(prompts_dir):
            print(f"\nContents of {prompts_dir}:")
            print(os.listdir(prompts_dir))

Successfully imported modules from rag-compliance directory


## Tools

In [15]:
tools_definitions()

[{'type': 'function',
  'function': {'name': 'agent_sql_analysis',
   'description': "Generates executable Microsoft SQL Server (MSSQL) queries based on user requests for database operations. The 'user_request' parameter can be any data type (string, list, dictionary, tuple, JSON object, etc.), including natural language, structured instructions, or outputs from other agents. The function should interpret the intent and generate an appropriate SQL query.",
   'parameters': {'type': 'object',
    'properties': {'user_request': {'type': 'string',
      'description': "The user's request, which can be any data type (string, list, dictionary, tuple, JSON object, etc.), including natural language, structured input, or the output from the router agent about what database operation to perform."},
     'identifier': {'type': 'string',
      'description': 'Optional: The customer ID or account number if specified in the request'}},
    'required': ['user_request']}}}]

In [ ]:
def agent_sql_analysis(user_request=None, identifier=None):
    pass

def agent_product_offerings(user_request=None, identifier=None):
    pass

def agent_file_validation(file_path=None, validation_type="standard"):
    pass

def agent_compare_files(baseline_data=None, comparison_data=None, comparison_type="standard"):
    pass
def agent_others(baseline_data=None, comparison_data=None, comparison_type="standard"):
    pass

## Router Agent

In [ ]:
# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

In [ ]:
# Create client
client = AzureOpenAI(
  # default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
  api_key=API_KEY,
  azure_endpoint=API_ENDPOINT,
  azure_deployment= AZURE_DEPLOYMENT,
  api_version=API_VERSION, 
)


In [ ]:

def generate_response(user_query):
    completions = client.chat.completions.create(
        model=MODEL,
        stream=True,
        
        messages=[
            {"role": "user", "content": user_query}
        ],
        # tools=tools_definitions(),
        # tool_choice="auto"
    )

    for chunk in completions:
        try:
            # print(chunk.choices[0].delta.content or "", end="")
            print(chunk.choices[0].delta.content or "", end="")
        except Exception:
            pass

print(generate_response("What makes AI agents AI agents?"))

2025/06/12 07:14:34 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Great question! The term **"AI agent"** refers to a specific kind of software system within the field of artificial intelligence. Here’s what makes an AI agent an AI agent:

---

### **1. Perception–Action Loop**

An AI agent **perceives** its environment (through sensors or input data), **processes** this information, and then **acts** upon the environment (through actuators or output actions). This cycle is central:

- **Perceive** → **Decide/Reason** → **Act** → (repeat)

### **2. Autonomy**

AI agents operate **autonomously**: they make decisions and take actions on their own, without constant human intervention.

### **3. Goal-Directed Behavior**

They are **goal-oriented**: AI agents have some objective(s) they aim to achieve (e.g., maximizing a reward, reaching a destination, providing an answer).

### **4. Adaptivity**

Many AI agents can **adapt** their behavior based on experience or changes in the environment. This reinforces their intelligence.

### **5. Rationality**

They

In [ ]:
def agent_router(user_query):
    completions = client.chat.completions.create(
        model=MODEL,
        # stream=True,
        messages=[
            {"role": "system", "content": prompt_agent_router()},
            {"role": "user", "content": user_query}
        ],
        tools=tools_definitions(),
        tool_choice="auto"
    )
    return completions


In [ ]:
# No tool call
response = agent_router("what is an ai agent?")

2025/06/12 08:30:24 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


In [ ]:
response

ChatCompletion(id='chatcmpl-Bh8gHu8yhQ71gOWaHjE9y66lBXB9w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='An AI agent is an autonomous or semi-autonomous computer program or system that perceives its environment, processes information, and takes actions to achieve specific goals or tasks. AI agents use artificial intelligence techniques, such as machine learning, reasoning, or natural language processing, to make decisions and adapt to new situations.\n\nKey characteristics of an AI agent:\n- Perceives: Receives inputs from its environment (e.g., sensors, user inputs, data).\n- Processes: Analyzes and interprets the received information using algorithms.\n- Acts: Executes actions or provides responses that affect the environment or accomplish tasks.\n- Goal-oriented: Designed to achieve specific objectives, often optimizing some measure of performance.\n\nExamples include: virtual assistants (like Siri or Alexa), self-driving cars,

In [ ]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='An AI agent is an autonomous or semi-autonomous computer program or system that perceives its environment, processes information, and takes actions to achieve specific goals or tasks. AI agents use artificial intelligence techniques, such as machine learning, reasoning, or natural language processing, to make decisions and adapt to new situations.\n\nKey characteristics of an AI agent:\n- Perceives: Receives inputs from its environment (e.g., sensors, user inputs, data).\n- Processes: Analyzes and interprets the received information using algorithms.\n- Acts: Executes actions or provides responses that affect the environment or accomplish tasks.\n- Goal-oriented: Designed to achieve specific objectives, often optimizing some measure of performance.\n\nExamples include: virtual assistants (like Siri or Alexa), self-driving cars, recommendation systems, chatbots, and diagnostic tools.', refusal=No

In [ ]:
print(response.choices[0].message.content)

An AI agent is an autonomous or semi-autonomous computer program or system that perceives its environment, processes information, and takes actions to achieve specific goals or tasks. AI agents use artificial intelligence techniques, such as machine learning, reasoning, or natural language processing, to make decisions and adapt to new situations.

Key characteristics of an AI agent:
- Perceives: Receives inputs from its environment (e.g., sensors, user inputs, data).
- Processes: Analyzes and interprets the received information using algorithms.
- Acts: Executes actions or provides responses that affect the environment or accomplish tasks.
- Goal-oriented: Designed to achieve specific objectives, often optimizing some measure of performance.

Examples include: virtual assistants (like Siri or Alexa), self-driving cars, recommendation systems, chatbots, and diagnostic tools.


In [ ]:
# tool call
response = agent_router("show me top 5 clients by account balance?")

In [ ]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HrsE2GBwvEBPKuD6AzGTmC3O', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balance from the customer_information table. Display their full name, account number, and balance, ordered by balance in descending order. Limit the results to 5 records."}', name='agent_sql_analysis'), type='function')]), content_filter_results={})

In [ ]:
output = response.choices[0].message
output

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HrsE2GBwvEBPKuD6AzGTmC3O', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balance from the customer_information table. Display their full name, account number, and balance, ordered by balance in descending order. Limit the results to 5 records."}', name='agent_sql_analysis'), type='function')])

In [ ]:
print(output.content)

None


In [ ]:
output.tool_calls[0]

ChatCompletionMessageToolCall(id='call_HrsE2GBwvEBPKuD6AzGTmC3O', function=Function(arguments='{"user_request":"Retrieve the top 5 clients with the highest account balance from the customer_information table. Display their full name, account number, and balance, ordered by balance in descending order. Limit the results to 5 records."}', name='agent_sql_analysis'), type='function')

In [ ]:
output.tool_calls[0].function.name

'agent_sql_analysis'

In [ ]:
output.tool_calls[0].function.arguments

'{"user_request":"Retrieve the top 5 clients with the highest account balance from the customer_information table. Display their full name, account number, and balance, ordered by balance in descending order. Limit the results to 5 records."}'

## agent_sql_analysis

In [ ]:
def agent_sql_analysis(user_input):
    """
    Processes SQL query-related requests.
    
    Args:
        user_input: String or JSON-serializable input from the user
        
    Returns:
        str: The generated response text
    """
    # Accepts any data type: string, dict, list, etc.
    # If not string, convert to JSON string for the LLM
    if not isinstance(user_input, str):
        user_input_serialized = json.dumps(user_input)
    else:
        user_input_serialized = user_input

    completions = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt_agent_sql_analysis()},
            {"role": "user", "content": user_input_serialized}
        ],
    )
    return completions.choices[0].message.content



In [ ]:
user_query = output.tool_calls[0].function.arguments
sql_output = agent_sql_analysis(user_query)

NameError: name 'output' is not defined

In [ ]:
print(sql_output)

```sql
SELECT TOP 5 
    full_name,
    account_number,
    balance
FROM [dbo].[customer_information] WITH (NOLOCK)
ORDER BY balance DESC;
```


In [ ]:
sql_results = execute_sql_with_pyodbc(sql_output)
display_sql_results(sql_results, max_query_preview=200, max_result_preview=100)


==================== QUERY #1 ====================
Status: success
Query: SELECT TOP 5 
    full_name,
    account_number,
    balance
FROM [dbo].[customer_information] WITH (NOLOCK)
ORDER BY balance DESC;
Rows: 5 | Columns: 3
Column names: full_name, account_number, balance
Result: [{"full_name":"Charles Cline","account_number":"5423480814","balance":49859.96},{"full_name":"Anthon...
--------------------------------------------------


In [ ]:
sql_results

[{'query': 'SELECT TOP 5 \n    full_name,\n    account_number,\n    balance\nFROM [dbo].[customer_information] WITH (NOLOCK)\nORDER BY balance DESC;',
  'status': 'success',
  'row_count': 5,
  'column_count': 3,
  'columns': ['full_name', 'account_number', 'balance'],
  'result': '[{"full_name":"Charles Cline","account_number":"5423480814","balance":49859.96},{"full_name":"Anthony Kelly","account_number":"4563140853","balance":49832.11},{"full_name":"Ruth Jackson","account_number":"5615994343","balance":49754.38},{"full_name":"Joseph Rush","account_number":"5867848730","balance":49650.18},{"full_name":"Christine Saunders","account_number":"1790890359","balance":49550.14}]'}]

# Chat Loops

In [ ]:
chat_history =[]

In [ ]:
import json

def routing_agent(user_request, chat_history):
    """Routes the user request to the appropriate function/tool, using chat history for context."""
    prompt = prompt_agent_router()

    messages = [{"role": "system", "content": prompt}]
    messages.extend(chat_history)

    # Ensure latest user message is not duplicated
    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_request:
        messages.append({"role": "user", "content": user_request})

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools_definitions(),
        tool_choice="auto"
    )
    return response

def ai_chatbot(user_input, chat_history):
    # Avoid duplicate user input
    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_input:
        chat_history.append({"role": "user", "content": user_input})

    response = routing_agent(user_input, chat_history)
    message = response.choices[0].message

    if hasattr(message, 'tool_calls') and message.tool_calls:
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        sql_results, product_offerings, validation_results = [], [], None

        if function_name == "agent_sql_analysis":
            sql_code = agent_sql_analysis(function_args["user_request"])
            sql_results = execute_sql_with_pyodbc(sql_code)

        elif function_name == "agent_product_offerings":
            product_offerings = [agent_product_offerings(function_args["user_request"])]

        elif function_name == "agent_file_validation":
            validation_results = agent_file_validation(function_args["file_path"])

        elif function_name == "agent_compare_files":
            validation_results = agent_compare_files(
                function_args["baseline_data"],
                function_args["comparison_data"],
                function_args.get("comparison_type", "standard")
            )

        else:
            assistant_msg = "I don't know how to handle that request."
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                chat_history.append({"role": "assistant", "content": assistant_msg})
            return chat_history

        # Process SQL results
        processed_sql_data = []
        for res in sql_results:
            if isinstance(res, dict) and res.get("status") == "success":
                try:
                    json_data = json.loads(res["result"])
                    processed_sql_data.append({
                        "query_info": f"Query returned {res['row_count']} rows with columns: {', '.join(res['columns'])}",
                        "data": json_data
                    })
                except json.JSONDecodeError:
                    processed_sql_data.append({
                        "query_info": f"Query returned {res['row_count']} rows",
                        "data": res["result"]
                    })
            else:
                processed_sql_data.append({
                    "query_info": f"Query failed with status: {res.get('status', 'unknown')}",
                    "data": res.get("result", "No data available")
                })

        # Build polish prompt
        if validation_results:
            polish_prompt = (
                "Please provide a clear, friendly response about the validation results: "
                f"{validation_results}. Format the response to be easy to understand."
            )
        elif sql_results and product_offerings:
            polish_prompt = (
                "The user asked for both data and product information. Provide a comprehensive response that:\n"
                f"1. Summarizes the data results: {processed_sql_data}\n"
                f"2. Describes the product offerings: {product_offerings}"
            )
        elif sql_results:
            polish_prompt = (
                "Summarize the following database query results in a clear, friendly way:\n"
                f"{processed_sql_data}"
            )
        elif product_offerings:
            polish_prompt = (
                "Present the following product information in a helpful way:\n"
                f"{product_offerings}"
            )
        else:
            assistant_msg = "I couldn't process your request. Please try again."
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                chat_history.append({"role": "assistant", "content": assistant_msg})
            return chat_history
        # Use polish_messages correctly
        polish_messages = [
            {"role": "system", "content": "You are a helpful banking assistant that provides clear, friendly responses to users."},
            {"role": "user", "content": polish_prompt}
        ]

        # polish_response = client.chat.completions.create(
        #     model=MODEL,
        #     messages=polish_messages
        # )

        polish_response = routing_agent(polish_prompt, chat_history)

        final_message = polish_response.choices[0].message.content
        if final_message and final_message.strip():
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != final_message:
                chat_history.append({"role": "assistant", "content": final_message})

    else:
        content = message.content or "I don't have a response for that."
        if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != content:
            chat_history.append({"role": "assistant", "content": content})

    return chat_history


In [ ]:
from IPython.display import clear_output, display, Markdown
import sys

chat_history = []

def inline_input(prompt=""):
    """Custom input function that displays inline in Jupyter notebook"""
    print(prompt, end='')
    sys.stdout.flush()
    return input()

def display_chat(chat_history):
    """Nicely formats and displays the full chat history"""
    clear_output(wait=True)
    for msg in chat_history:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            display(Markdown(f"**You:** {content}"))
        elif role == "assistant":
            display(Markdown(f"**Assistant:** {content}"))

# Main loop
while True:
    display_chat(chat_history)
    user_input = inline_input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break

    # Process input and update chat history
    chat_history = ai_chatbot(user_input, chat_history)


**You:** which client has the highest balance? show the top five

**Assistant:** Here are the top five clients with the highest account balances:

1. Charles Cline – Account: 5423480814 – Balance: $49,859.96
2. Anthony Kelly – Account: 4563140853 – Balance: $49,832.11
3. Ruth Jackson – Account: 5615994343 – Balance: $49,754.38
4. Joseph Rush – Account: 5867848730 – Balance: $49,650.18
5. Christine Saunders – Account: 1790890359 – Balance: $49,550.14

Charles Cline currently holds the highest balance among all clients.

**You:** show me the bottom five clients by age and balance

**Assistant:** Here’s a clear summary of the query results:

Bottom Five Clients by Age:
All five youngest clients are 18 years old. Their account balances vary, with the highest among them being $37,808.13 and the lowest at $17,945.07. Here are their details:

- Seth Rose (Account: 622207409) – Balance: $37,808.13
- Joshua Hopkins (Account: 4644941168) – Balance: $38,274.70
- Carol Johnson (Account: 1716850059) – Balance: $18,510.14
- Jeremy Adams (Account: 7805186430) – Balance: $17,945.07
- Jamie Cabrera (Account: 9438203299) – Balance: $24,435.26

Bottom Five Clients by Balance:
These clients have the lowest account balances in the database, ranging from $158.62 to $487.28. Their ages vary from 21 to 57. Here are their details:

- Christy Adams (Account: 2310140863, Age: 40) – Balance: $158.62
- Tiffany Luna (Account: 6282508959, Age: 52) – Balance: $175.85
- Mr. John Romero MD (Account: 330207984, Age: 21) – Balance: $237.72
- Charles Jackson (Account: 3389597397, Age: 25) – Balance: $384.45
- Steven Kirk (Account: 32574501, Age: 57) – Balance: $487.28

In summary, the youngest clients are all 18, with a wide range of balances, while the lowest balances are found across a wider age range.

You: Exiting chat.


In [ ]:
chat_history

[{'role': 'user',
  'content': 'which client has the highest balance? show the top five'},
 {'role': 'assistant',
  'content': 'Here are the top five clients with the highest account balances:\n\n1. Charles Cline – Account: 5423480814 – Balance: $49,859.96\n2. Anthony Kelly – Account: 4563140853 – Balance: $49,832.11\n3. Ruth Jackson – Account: 5615994343 – Balance: $49,754.38\n4. Joseph Rush – Account: 5867848730 – Balance: $49,650.18\n5. Christine Saunders – Account: 1790890359 – Balance: $49,550.14\n\nCharles Cline currently holds the highest balance among all clients.'},
 {'role': 'user',
  'content': 'show me the bottom five clients by age and balance'},
 {'role': 'assistant',
  'content': 'Here’s a clear summary of the query results:\n\nBottom Five Clients by Age:\nAll five youngest clients are 18 years old. Their account balances vary, with the highest among them being $37,808.13 and the lowest at $17,945.07. Here are their details:\n\n- Seth Rose (Account: 622207409) – Balance:

# MLFlow Integration

In [ ]:
# utils/mlflow_logger.py
import mlflow
import json

def start_chat_run(user_input):
    mlflow.set_experiment("Chatbot_Tracing")
    mlflow.start_run(nested=True)
    mlflow.log_param("user_input", user_input)

def log_router_response(message):
    mlflow.log_text(str(message), "router_response.json")

def log_sql_code(sql_code):
    mlflow.log_text(sql_code, "generated_sql.sql")

def log_sql_results(results):
    mlflow.log_text(json.dumps(results, indent=2), "sql_results.json")

def log_product_info(info):
    mlflow.log_text(json.dumps(info, indent=2), "product_info.json")

def log_validation_result(result, filename="validation_result.txt"):
    mlflow.log_text(str(result), filename)

def log_final_response(response):
    mlflow.log_text(response, "final_response.txt")

def end_chat_run():
    mlflow.end_run()


In [ ]:
from utils.mlflow_logger import (
    start_chat_run, log_router_response, log_sql_code, log_sql_results,
    log_product_info, log_validation_result, log_final_response, end_chat_run
)


In [ ]:
def ai_chatbot(user_input, chat_history):
    if not chat_history or chat_history[-1]["role"] != "user" or chat_history[-1]["content"] != user_input:
        chat_history.append({"role": "user", "content": user_input})

    start_chat_run(user_input)

    try:
        response = routing_agent(user_input, chat_history)
        message = response.choices[0].message
        log_router_response(message)

        if hasattr(message, 'tool_calls') and message.tool_calls: # if the router invoked a tool, do the following
            tool_call = message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function Name: {function_name}")
            print(f"\nFunction Arguments:\n {function_args}")

            sql_results, product_offerings, validation_results = [], [], None

            if function_name == "agent_sql_analysis":
                sql_code = agent_sql_analysis(function_args["user_request"])
                log_sql_code(sql_code)
                print(f"Generated SQL Code:\n {sql_code}") 
                sql_results = execute_sql_with_pyodbc(sql_code)
                log_sql_results(sql_results)

            elif function_name == "agent_product_offerings":
                product_info = agent_product_offerings(function_args["user_request"])
                product_offerings = [product_info]
                log_product_info(product_info)

            elif function_name == "agent_file_validation":
                validation_results = agent_file_validation(function_args["file_path"])
                log_validation_result(validation_results, "file_validation.txt")

            elif function_name == "agent_compare_files":
                validation_results = agent_compare_files(
                    function_args["baseline_data"],
                    function_args["comparison_data"],
                    function_args.get("comparison_type", "standard")
                )
                log_validation_result(validation_results, "file_comparison.txt")

            else:
                assistant_msg = "I don't know how to handle that request."
                if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                    chat_history.append({"role": "assistant", "content": assistant_msg})
                return chat_history

            # Process SQL results
            processed_sql_data = []
            for res in sql_results:
                if isinstance(res, dict) and res.get("status") == "success":
                    try:
                        json_data = json.loads(res["result"])
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows with columns: {', '.join(res['columns'])}",
                            "data": json_data
                        })
                    except json.JSONDecodeError:
                        processed_sql_data.append({
                            "query_info": f"Query returned {res['row_count']} rows",
                            "data": res["result"]
                        })
                else:
                    processed_sql_data.append({
                        "query_info": f"Query failed with status: {res.get('status', 'unknown')}",
                        "data": res.get("result", "No data available")
                    })

            # Build polish prompt
            if validation_results:
                polish_prompt = (
                    "Please provide a clear, friendly response about the validation results: "
                    f"{validation_results}. Format the response to be easy to understand."
                )
            elif sql_results and product_offerings:
                polish_prompt = (
                    "The user asked for both data and product information. Provide a comprehensive response that:\n"
                    f"1. Summarizes the data results: {processed_sql_data}\n"
                    f"2. Describes the product offerings: {product_offerings}"
                )
            elif sql_results:
                polish_prompt = (
                    "Summarize the following database query results in a clear, friendly way:\n"
                    f"{processed_sql_data}"
                )
            elif product_offerings:
                polish_prompt = (
                    "Present the following product information in a helpful way:\n"
                    f"{product_offerings}"
                )
            else:
                assistant_msg = "I couldn't process your request. Please try again."
                if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != assistant_msg:
                    chat_history.append({"role": "assistant", "content": assistant_msg})
                return chat_history

            # Use polish_messages correctly
            polish_messages = [
                {"role": "system", "content": "You are a helpful banking assistant that provides clear, friendly responses to users."},
                {"role": "user", "content": polish_prompt}
            ]

            # polish_response = client.chat.completions.create(
            #     model=MODEL,
            #     messages=polish_messages
            # )
            polish_response = routing_agent(polish_prompt, chat_history)

            final_message = polish_response.choices[0].message.content
            log_final_response(final_message)

            if final_message and final_message.strip():
                if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != final_message:
                    chat_history.append({"role": "assistant", "content": final_message})

        else:
            content = message.content or "I don't have a response for that."
            log_final_response(content)
            if not chat_history or chat_history[-1]["role"] != "assistant" or chat_history[-1]["content"] != content:
                chat_history.append({"role": "assistant", "content": content})

    finally:
        end_chat_run()

    return chat_history


In [ ]:
from IPython.display import clear_output, display, Markdown
import sys

chat_history = []

def inline_input(prompt=""):
    """Custom input function that displays inline in Jupyter notebook"""
    print(prompt, end='')
    sys.stdout.flush()
    return input()

def display_chat(chat_history):
    """Nicely formats and displays the full chat history"""
    clear_output(wait=True)
    for msg in chat_history:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            display(Markdown(f"**You:** {content}"))
        elif role == "assistant":
            display(Markdown(f"**Assistant:** {content}"))

# Main loop
while True:
    display_chat(chat_history)
    user_input = inline_input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break

    # Process input and update chat history
    chat_history = ai_chatbot(user_input, chat_history)


You: Exiting chat.


In [ ]:
chat_history = []
user_input = " which client has the highest account balance? and also show me which month had the highest amount."
ai_chatbot(user_input, chat_history)

Function Name: agent_sql_analysis

Function Arguments:
 {'user_request': 'Find the customer with the highest account balance from the customer_information table. Include their full name and balance amount.'}


TypeError: write() argument must be str, not None

In [ ]:
sql_code = """
 ```sql
SELECT TOP 1
    YEAR(transaction_date) AS transaction_year,
    MONTH(transaction_date) AS transaction_month,
    SUM(amount) AS total_amount
FROM [dbo].[transaction_history] WITH (NOLOCK)
WHERE status = 'Completed'
GROUP BY YEAR(transaction_date), MONTH(transaction_date)
ORDER BY total_amount DESC;
```"""
execute_sql_with_pyodbc(sql_code)

[{'query': "SELECT TOP 1\n    YEAR(transaction_date) AS transaction_year,\n    MONTH(transaction_date) AS transaction_month,\n    SUM(amount) AS total_amount\nFROM [dbo].[transaction_history] WITH (NOLOCK)\nWHERE status = 'Completed'\nGROUP BY YEAR(transaction_date), MONTH(transaction_date)\nORDER BY total_amount DESC;",
  'result': "Query validation failed: Suspicious SQL pattern detected: '.*?'.*?;",
  'status': 'validation_error'}]